# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Токенизация и классификация рекуррентными нейронными сетями

### Оценивание и штрафы

Максимально допустимая оценка за работу — 13 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

Это задание посвящено решению задачи классификации тональности текста с помощью рекуррентных нейронных сетей (и сверточных). В отличие от предыдущей домашки, в этом задании не будет шаблонов кода, вам придется написать все самостоятельно, поэтому постарайтесь хорошо организовать код. Мы крайне рекомендуем реализовывать все модели и вспомогательные функции в отдельных файлах, а затем их импортировать. Иначе вы рискуете превратить ноутбук в кашу.

Обучение не должно занимать много времени, однако для ускорения мы советуем использовать различные приемы, такие как mixed-precision ([ссылка](https://huggingface.co/docs/accelerate/package_reference/accelerator) и [ссылка](https://pytorch.org/docs/stable/amp.html)) и torch.compile (для torch >= 2.0, [ссылка](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)). Так же мы настоятельно рекомендуем логировать все графики обучения с помощью платформы [wandb](https://docs.wandb.ai/quickstart).

Все модели нужно будет реализовывать на pytorch (считать градиенты руками не надо). При обучении разных моделей используйте одинаковые параметры оптимизатора, базмер батча, а так же учите либо одинаковое число эпох, либо до сходимости. В общем, постарайтесь, чтобы сравнение всегда было максимально честным.


__Мягкий дедлайн: 03.11.23 23:59__

__Жесткий дедлайн: 07.11.23 23:59__

### Данные и токенизация

Мы будем обучать модель на задачу классификации тональности текста. Данные взяты с платформы IMDb и содержат положительные и отрицательные отзывы о фильмах, примерно по 5к текстов каждого класса. Вы можете найти их в папке `datasets/IMDb`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'gpu'
device

'cuda'

__Задание 1 (2 балла)__
Как всегда, для работы с текстом нам нужно как-то его токенизировать. В этом задании вам предстоит заняться этим. Выберите один из трех методов токенизации, разобранных на лекции: __BPE__, __WordPiece__, __Unigram__. Реализуйте его самостоятельно без использования специализированных библиотек (например, huggingface). Ваш токенизатор должен иметь интерфейс токенизатора из huggingface. То есть он должен иметь метод `encode`, возвращающий словарь с полями `input_ids` и `attention_mask`, а так же метод `decode`, принимающий последовательности токенов и возвращающий соответствующие им тексты. Ограничьте размер словаря 30000 токенами. Ваш токенизатор не обязан добавлять токены начала и конца последовательности, потому что они не обязательны в задаче классификации. Однако при желании вы можете их добавить. Все тонкости реализации, такие как прочие аргументы функций, типы переменных и так далее остаются на ваше усмотрение.

In [ ]:
from BPETokenizer import BPETokenizer

tokenizer = BPETokenizer()
files = [f"datasets/IMDb/{split}/{kind}" for split in ["train"] for kind in ["negative", "positive"]]
allLines = []

for fileName in files:
    with open(fileName, 'r') as file:
        allLines += [str.lower().replace('\n', '').replace('<br /><br />', '') for str in file.readlines()]

tokenizer.fit(allLines)

In [ ]:
import pickle

In [ ]:
# Сохранение tokenizer в файл
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

In [ ]:
# Загрузка tokenizer из файла
with open('tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

In [ ]:
tokenizer.ids2tokens[20000:20010]

['reatl',
 'rease',
 'rcy',
 'razi',
 'ranci',
 'rainy',
 'radua',
 'radu',
 'quisi',
 'polo']

In [ ]:
print(allLines[0])
encoded = tokenizer.encode([allLines[0]])['input_ids'][0]
print(encoded)
decoded = tokenizer.decode(encoded)
print(decoded)
print([tokenizer.ids2tokens[id] for id in encoded])

__Задание 1 (0 баллов)__
Прочитайте датасет и сложите все в `DataLoader`. Вы можете предобрабатывать тексты дополнительно как вам хочется.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class RecommendationsDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.LongTensor(data)
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data[index]
        label = self.labels[index]
        return text, label

In [ ]:
dim_size = 32

with open("datasets/IMDb/train/negative", 'r') as file:
    trainNegativeLines = [str.lower().replace('\n', '').replace('<br /><br />', '') for str in file.readlines()]
    for line in trainNegativeLines:
        while '  ' in line:
            line = line.replace('  ', ' ')
with open("datasets/IMDb/train/positive", 'r') as file:
    trainPositiveLines = [str.lower().replace('\n', '').replace('<br /><br />', '') for str in file.readlines()]
    for line in trainPositiveLines:
        while '  ' in line:
            line = line.replace('  ', ' ')

with open("datasets/IMDb/test/negative", 'r') as file:
    testNegativeLines = [str.lower().replace('\n', '').replace('<br /><br />', '') for str in file.readlines()]
    for line in testNegativeLines:
        while '  ' in line:
            line = line.replace('  ', ' ')
with open("datasets/IMDb/test/positive", 'r') as file:
    testPositiveLines = [str.lower().replace('\n', '').replace('<br /><br />', '') for str in file.readlines()]
    for line in testPositiveLines:
        while '  ' in line:
            line = line.replace('  ', ' ')

trainLen = len(trainNegativeLines) + len(trainPositiveLines)
tokenizedLines = tokenizer.encode(trainNegativeLines + trainPositiveLines + testNegativeLines + testPositiveLines)['input_ids']
trainLabels = [0] * len(trainNegativeLines) + [1] * len(trainPositiveLines)
trainDataset = RecommendationsDataset(tokenizedLines[:trainLen], trainLabels)
trainDataloader = DataLoader(trainDataset, batch_size=64, shuffle=True)

testLabels = [0] * len(testNegativeLines) + [1] * len(testPositiveLines)
testDataset = RecommendationsDataset(tokenizedLines[trainLen:], testLabels)
testDataloader = DataLoader(testDataset, batch_size=64, shuffle=False)

In [ ]:
print(len(trainDataset))
print(len(testDataset))
print(len(tokenizedLines[0]))

9788
9763
493


In [ ]:
from tqdm import tqdm

def accuracy_score(y_pred, y_true):
    predictions = (y_pred >= 0.5) * 1
    return sum(predictions == y_true) / len(predictions)

def train(model, trainDataloader, testDataloader, optim, epochs=30, criterion=torch.nn.BCELoss()):
    for i in range(epochs):
        trainLosses = []
        trainAccs = []
        allLen = 0
        for X_batch, Y_batch in tqdm(trainDataloader):
            optim.zero_grad()
            y_predict = model.forward(X_batch.to(device)).squeeze()
            loss = criterion(y_predict, Y_batch.to(device))
            loss.backward()
            optim.step()
            # statistics
            trainLosses.append(loss.item() * len(X_batch))
            trainAccs.append(accuracy_score(y_predict, Y_batch.to(device)) * len(X_batch))
            allLen += len(X_batch)
        print(f'Epoch {i}: TrainLoss = {sum(trainLosses)/allLen}')
        print(f'TrainAcc = {sum(trainAccs)/allLen}')
        testLoss, testAcc = evaluate(model, testDataloader, criterion)
        print(f'TestLoss = {testLoss}')
        print(f'TestAcc = {testAcc}')

def evaluate(model, testDataloader, criterion):
    testLosses = []
    testAccs = []
    allLen = 0
    with torch.no_grad():
        for X_batch, Y_batch in tqdm(testDataloader):
            y_predict = model.forward(X_batch.to(device)).squeeze()
            loss = criterion(y_predict,Y_batch.to(device))
            testLosses.append(loss.item() * len(X_batch))
            testAccs.append(accuracy_score(y_predict, Y_batch.to(device)) * len(X_batch))
            allLen += len(X_batch)
    meanLoss = sum(testLosses)/allLen
    meanAcc = sum(testAccs)/allLen
    return meanLoss, meanAcc

## Сверточные нейронные сети

![image.png](attachment:a19099b8-00da-4f48-ae06-5a72c7d3b5a5.png)

__Задание 3 (2 балла)__ В качестве бейзлайна для решения задачи реализуйте и обучите сверточную нейронную сеть с одномерными свертками (`torch.nn.Conv1d`). Число сверточных слоев можете выбрать по своему усмотрению, однако не делайте сеть слишком большой. Это задание требуется для установки бейзлайна, поэтому лучше не тратить много времени на обучение и подгонку параметров. У вас должна получиться точность классификации на тестовой выборке около 80% (больше тоже хорошо).

In [ ]:
from models import ConvNN

convModel = ConvNN(words_count=493,dim_hidden=dim_size).to(device)
convOptim = torch.optim.Adam(convModel.parameters(), lr=3e-4)
train(convModel, trainDataloader, testDataloader, convOptim, n_epochs=30)

100%|██████████| 153/153 [00:07<00:00, 20.16it/s]


Epoch 0: TrainLoss = 0.6935907584285074
TrainAcc = 0.4978545308113098


100%|██████████| 153/153 [00:00<00:00, 521.56it/s]


TestLoss = 0.6933564984150064
TestAcc = 0.5058895945549011


100%|██████████| 153/153 [00:00<00:00, 193.24it/s]


...

TestLoss = 0.6515915741518019
TestAcc = 0.8150159120559692


100%|██████████| 153/153 [00:00<00:00, 243.49it/s]


Epoch 29: TrainLoss = 0.09746299805077768
TrainAcc = 0.967409074306488


100%|██████████| 153/153 [00:00<00:00, 553.83it/s]

TestLoss = 0.6774696500089146
TestAcc = 0.8160401582717896


## Рекуррентные нейронные сети

![image.png](attachment:e0b5ba38-0b1d-4d2a-a685-f1a003e05324.png)

В этой секции вам предстоит реализовать два вида рекуррентных нейронных сетей: RNN, LSTM. Вот они с слева направо. Начнем с RNN.

__Задание 4 (1.5 балла)__ Реализуйте классическую рекуррентную нейронную сеть с одним слоем. Как вы знаете из лекции, такая сеть плохо выучивает долгосточные зависимости в данных, поэтому работают плохо с длинными текстами. Обучите реализованную RNN и проверьте, так ли это, замерив точность на тестовой выборке.

In [ ]:
from models import RNN

rnnModel = RNN(dim_embedding=dim_size, dim_hidden=64, device=device).to(device)
rnnOptim = torch.optim.Adam(rnnModel.parameters(), lr=3e-4)
train(rnnModel, trainDataloader, testDataloader, rnnOptim, epochs=20)

100%|██████████| 153/153 [01:16<00:00,  2.01it/s]


Epoch 0: TrainLoss = 0.6942532409635621
TrainAcc = 0.49969351291656494


100%|██████████| 153/153 [00:21<00:00,  7.15it/s]


TestLoss = 0.6943739070812235
TestAcc = 0.49400800466537476


100%|██████████| 153/153 [01:16<00:00,  2.00it/s]


...

TestLoss = 0.662799018731948
TestAcc = 0.6207108497619629


100%|██████████| 153/153 [01:15<00:00,  2.01it/s]


Epoch 19: TrainLoss = 0.5557696955468053
TrainAcc = 0.7205762267112732


100%|██████████| 153/153 [00:22<00:00,  6.70it/s]

TestLoss = 0.6723909830993333
TestAcc = 0.6303390264511108


Дообучим модель

In [ ]:
train(rnnModel, trainDataloader, testDataloader, rnnOptim, epochs=10)

100%|██████████| 153/153 [01:15<00:00,  2.02it/s]


Epoch 0: TrainLoss = 0.5382784831314414
TrainAcc = 0.7347772717475891


100%|██████████| 153/153 [00:21<00:00,  7.04it/s]


TestLoss = 0.6500459776529826
TestAcc = 0.6484687328338623


100%|██████████| 153/153 [01:16<00:00,  2.00it/s]


...

TestLoss = 0.635145000871498
TestAcc = 0.6928198337554932


100%|██████████| 153/153 [01:17<00:00,  1.98it/s]


Epoch 9: TrainLoss = 0.3998641362714436
TrainAcc = 0.8307110667228699


100%|██████████| 153/153 [00:22<00:00,  6.93it/s]

TestLoss = 0.6354890530267895
TestAcc = 0.6947659850120544


*Модель предсказывает на 10% хуже, чем сверточная сеть. Действительно, долгосрочные зависимости ей улавливать тяжело.*

__Задание 5 (2.5 балла)__
В предыдущем задании скорее всего качество RNN у вас должно было получиться хуже, чем у CNN. Попробуем исправить это, добавив в модель вектор памяти. Реализуйте модель LSTM и сравните ее качество с RNN. Напомним, что скрытые состояния LSTM считаются по следующим формулам:
$$
\begin{gathered}
f_t=\sigma\left(W_f \cdot\left[h_{t-1}, x_t\right]+b_f\right) \\
i_t=\sigma\left(W_i \cdot\left[h_{t-1}, x_t\right]+b_i\right) \\
o_t=\sigma\left(W_o \cdot\left[h_{t-1}, x_t\right]+b_o\right) \\
\tilde{C}_t=\tanh \left(W_c \cdot\left[h_{t-1}, x_t\right]+b_c\right) \\
C_t=f_t \odot C_{t-1}+i_t \odot \tilde{C}_t \\
h_t=o_t \odot \tanh \left(C_t\right)
\end{gathered}
$$

In [ ]:
from models import LSTM

lstmModel = LSTM(dim_embedding=dim_size, dim_hidden=64, device=device).to(device)
lstmOptim = torch.optim.Adam(lstmModel.parameters(), lr=3e-4)
train(lstmModel, trainDataloader, testDataloader, lstmOptim, epochs=30)

100%|██████████| 153/153 [03:11<00:00,  1.25s/it]


Epoch 0: TrainLoss = 0.7276189372059175
TrainAcc = 0.5084797739982605


100%|██████████| 153/153 [00:49<00:00,  3.06it/s]


TestLoss = 0.6928148559770366
TestAcc = 0.5242241024971008


100%|██████████| 153/153 [02:47<00:00,  1.10s/it]


...

TestLoss = 0.5834472307224298
TestAcc = 0.8162450194358826


100%|██████████| 153/153 [02:35<00:00,  1.01s/it]


Epoch 29: TrainLoss = 0.10365928821554951
TrainAcc = 0.9711892008781433


100%|██████████| 153/153 [00:42<00:00,  3.64it/s]

TestLoss = 0.593229417678556
TestAcc = 0.8206493854522705


***Как видим, модель LSTM дааёт лучше результат, чем RNN: 80% против 70%. Сверочная сеть для классификации текстов самая приятная и по эффективности, и по времени.***

__Бонус (1 балл)__ Проверьте, на что влияет каждый гейт в LSTM. Попробуйте отключать каждый из них по очереди (выход гейта всегда 1) и посмотрите, при отключении каких из них точность падает сильнее. Совпадает ли результат с вашими ожиданиями?

*Предположение: gate o повлияет на точность меньше всего, он кажется бесполезным, гейты f и i отразятся на результате сильнее, отключение i будет самым фатальным, так как в контекст будут попадать все векторы без разбору.*

In [ ]:
cache_o = lstmModel.Woh.data, lstmModel.boh.data
lstmModel.Woh.data = torch.zeros(lstmModel.Woh.shape).to(device)
lstmModel.boh.data = torch.ones(lstmModel.boh.shape).to(device)
loss, test_acc = evaluate(lstmModel, testDataloader, torch.nn.BCELoss())
print(f'Loss without o: {loss}   ||||   Accuracy without o: {test_acc.item()}')
lstmModel.Woh.data, lstmModel.boh.data = cache_o

100%|██████████| 153/153 [00:48<00:00,  3.15it/s]

Loss without o: 1.67920076013112   ||||   Accuracy without o: 0.5410222411155701


In [ ]:
cache_i = lstmModel.Wih.data, lstmModel.bih.data
lstmModel.Wih.data = torch.zeros(lstmModel.Wih.shape).to(device)
lstmModel.bih.data = torch.ones(lstmModel.bih.shape).to(device)
loss, test_acc = evaluate(lstmModel, testDataloader, torch.nn.BCELoss())
print(f'Loss without i: {loss}   ||||   Accuracy without i: {test_acc.item()}')
lstmModel.Wih.data, lstmModel.bih.data = cache_i

100%|██████████| 153/153 [00:43<00:00,  3.53it/s]

Loss without i: 2.8646949304108076   ||||   Accuracy without i: 0.4994366466999054


In [ ]:
cache_f = lstmModel.Wfh.data, lstmModel.bfh.data
lstmModel.Wfh.data = torch.zeros(lstmModel.Wfh.shape).to(device)
lstmModel.bfh.data = torch.ones(lstmModel.bfh.shape).to(device)
loss, test_acc = evaluate(lstmModel, testDataloader, torch.nn.BCELoss())
print(f'Loss without f: {loss}   ||||   Accuracy without f: {test_acc.item()}')
lstmModel.Wfh.data, lstmModel.bfh.data = cache_f

100%|██████████| 153/153 [00:46<00:00,  3.28it/s]

Loss without f: 0.757747303947469   ||||   Accuracy without f: 0.6685445308685303


*Оказалось, что без вектора i модель вообще не может существовать (как предполагалось), однако и gate o критичен для модели, он выполняет преобразование контекста в hidden вектор. Вектор f, отвечающий за забываемость контекста, не так важен, и без него модель ещё может работать, просто сохраняется вообще весь предыдущий контекст, а не выборочно. Но точность при отключении любого из гейтов значительно падает.*

__Задание 6 (1 балл): Многослойная рекуррентная сеть.__
Часто увеличение число слоев в рекуррентных нейронных сетях помогает улучшить способность модели извлекать информацию из текста. Дополнительный слой – это еще одна LSTM поверх выходов предыдущей. Добавьте в вашу модель возможность создания сети с произвольным числом слоев. Обучите двухслойную LSTM и сравните качество с однослойной версией. Подтвердилась ли теория?

![image.png](attachment:b09010ac-64d0-4606-a033-ec6a4a1f3e87.png)


In [ ]:
from models import LSTMnLayers

lstm2LModel = LSTMnLayers(dim_embedding=dim_size, dim_hidden=64, n_layers=2, device=device).to(device)
lstm2LOptim = torch.optim.Adam(lstm2LModel.parameters(), lr=3e-4)
train(lstm2LModel, trainDataloader, testDataloader, lstm2LOptim, epochs=10)

100%|██████████| 153/153 [04:56<00:00,  1.94s/it]


Epoch 0: TrainLoss = 0.7111695969791669
TrainAcc = 0.5085819363594055


100%|██████████| 153/153 [01:18<00:00,  1.95it/s]


TestLoss = 0.6919295886531663
TestAcc = 0.5079381465911865


100%|██████████| 153/153 [04:53<00:00,  1.92s/it]


...

TestLoss = 0.5090879647443374
TestAcc = 0.7641093730926514


100%|██████████| 153/153 [04:57<00:00,  1.95s/it]


Epoch 9: TrainLoss = 0.3973173891518815
TrainAcc = 0.8299959301948547


100%|██████████| 153/153 [01:17<00:00,  1.98it/s]

TestLoss = 0.5148865744054874
TestAcc = 0.7573491930961609


In [ ]:
train(lstm2LModel, trainDataloader, testDataloader, lstm2LOptim, epochs=5)

100%|██████████| 153/153 [04:54<00:00,  1.93s/it]


Epoch 0: TrainLoss = 0.37223726551192704
TrainAcc = 0.8452186584472656


100%|██████████| 153/153 [01:17<00:00,  1.98it/s]


TestLoss = 0.4858975432134732
TestAcc = 0.7807026505470276


100%|██████████| 153/153 [04:56<00:00,  1.94s/it]


...

TestLoss = 0.4677857994166332
TestAcc = 0.7965789437294006


100%|██████████| 153/153 [04:56<00:00,  1.94s/it]


Epoch 4: TrainLoss = 0.2872161137287988
TrainAcc = 0.8880261778831482


100%|██████████| 153/153 [01:15<00:00,  2.02it/s]

TestLoss = 0.5031414609408141
TestAcc = 0.798627495765686


In [ ]:
train(lstm2LModel, trainDataloader, testDataloader, lstm2LOptim, epochs=5)

100%|██████████| 153/153 [04:55<00:00,  1.93s/it]


Epoch 0: TrainLoss = 0.26894116672657925
TrainAcc = 0.899162232875824


100%|██████████| 153/153 [01:15<00:00,  2.02it/s]


TestLoss = 0.4719444356723654
TestAcc = 0.8112260699272156


100%|██████████| 153/153 [04:55<00:00,  1.93s/it]


...

TestLoss = 0.45250675911592264
TestAcc = 0.8170644044876099


100%|██████████| 153/153 [04:59<00:00,  1.96s/it]


Epoch 4: TrainLoss = 0.2095766199770196
TrainAcc = 0.9279730319976807


100%|██████████| 153/153 [01:17<00:00,  1.97it/s]

TestLoss = 0.4745088768984638
TestAcc = 0.8204445242881775


*Видим, что качество по сравнению с обычной LSTM не выросло, либо выросло незначитально.*

__Задание 6 (1 балл): Двунаправленная рекуррентная сеть.__ Для некоторых задач классификации может помочь смотреть на последовательность слева направо и справа налево одновременно. Рекуррентные сети с такой способностью называются двунаправленными (bidirectional). Реализуйте данный функционал в вашей LSTM модели. Заметьте, что такая модификация увеличивает число параметров модели так же, как и добавление еще одного слоя. Какая из этих двух модификаций оказывается лучше?

__ВАЖНО:__ Подумайте о том, как лучше агрегировать выходы разнонаправленных моделей. Попробуйте разные способы.

![image.png](attachment:38d1abab-a2af-403f-bc0d-ec408b9bfd01.png)

In [ ]:
from models import LSTMbiDir

lstmBiDirModel = LSTMbiDir(dim_embedding=dim_size, dim_hidden=64, device=device).to(device)
lstmBiDirOptim = torch.optim.Adam(lstmBiDirModel.parameters(), lr=3e-4)
train(lstmBiDirModel, trainDataloader, testDataloader, lstmBiDirOptim, epochs=20)

100%|██████████| 153/153 [05:32<00:00,  2.18s/it]


Epoch 0: TrainLoss = 0.6914066407550925
TrainAcc = 0.5254393219947815


100%|██████████| 153/153 [01:33<00:00,  1.63it/s]


TestLoss = 0.688475683339121
TestAcc = 0.5432756543159485


100%|██████████| 153/153 [05:16<00:00,  2.07s/it]


...

TestLoss = 0.5105712092168188
TestAcc = 0.8069241046905518


100%|██████████| 153/153 [05:03<00:00,  1.98s/it]


Epoch 19: TrainLoss = 0.21765293786854076
TrainAcc = 0.9210257530212402


100%|██████████| 153/153 [01:21<00:00,  1.88it/s]

TestLoss = 0.4403835864972106
TestAcc = 0.8199324011802673


***Итог: Обе модификации не улучшили качество модели и показали примерно равный результат, однако просто LSTM обучается быстрее. Среди всех моделей сверточные сети предпочтительней благодаря своей скорости, ведь accuracy примерно одинаковый, однако LSTM оказался чуть более эффективным (~2% accuracy + выигрыш по лоссу)***